In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

In [3]:
train = pd.read_csv('train.csv')
train

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
18829,ANSL_51738,2017-01-26 00:00:00,2018-03-09 15:35:00,2.0,Tricolor,0.44,27.36,0,1,0.0,2
18830,ANSL_59900,2016-06-18 00:00:00,2017-07-09 08:37:00,NaN,Brown,0.73,14.25,15,4,2.0,4
18831,ANSL_53210,2010-07-21 00:00:00,2018-08-22 14:27:00,0.0,Calico Point,0.99,28.13,13,9,1.0,1
18832,ANSL_63468,2017-05-12 00:00:00,2018-02-08 14:05:00,0.0,Tan,0.55,44.82,13,9,1.0,2


In [4]:
train.shape

(18834, 11)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
 10  pet_category    18834 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 1.6+ MB


In [6]:
train.isna().sum()

pet_id               0
issue_date           0
listing_date         0
condition         1477
color_type           0
length(m)            0
height(cm)           0
X1                   0
X2                   0
breed_category       0
pet_category         0
dtype: int64

In [7]:
# заполняем пустые значения в "condition" средним

mean_class = train['condition'].mean()
print(round(mean_class))

train['condition'] = train['condition'].fillna(mean_class)
print(train.isna().sum())

1
pet_id            0
issue_date        0
listing_date      0
condition         0
color_type        0
length(m)         0
height(cm)        0
X1                0
X2                0
breed_category    0
pet_category      0
dtype: int64


In [ ]:
for i in train.columns:
  d = dict((x, list(train[i]).count(x)) for x in set(list(train[i])) if list(train[i]).count(x) > 1)
  print(d)
  
# повторяющиеся элементы в столбцах
# (просто нужно)

{}
{'2015-04-30 00:00:00': 5, '2017-02-25 00:00:00': 7, '2015-04-09 00:00:00': 6, '2017-11-26 00:00:00': 9, '2013-11-28 00:00:00': 2, '2010-08-01 00:00:00': 2, '2017-10-10 00:00:00': 11, '2008-09-27 00:00:00': 2, '2017-08-25 00:00:00': 9, '2016-04-20 00:00:00': 5, '2018-09-23 00:00:00': 3, '2015-10-05 00:00:00': 8, '2012-03-01 00:00:00': 2, '2017-10-23 00:00:00': 10, '2014-05-23 00:00:00': 2, '2019-01-22 00:00:00': 5, '2018-06-13 00:00:00': 8, '2007-04-09 00:00:00': 2, '2014-11-29 00:00:00': 5, '2011-05-21 00:00:00': 2, '2012-03-18 00:00:00': 4, '2014-11-28 00:00:00': 4, '2017-09-16 00:00:00': 16, '2014-09-29 00:00:00': 3, '2012-02-01 00:00:00': 2, '2012-07-11 00:00:00': 2, '2016-05-01 00:00:00': 13, '2017-09-07 00:00:00': 6, '2016-06-25 00:00:00': 18, '2015-02-23 00:00:00': 6, '2016-02-29 00:00:00': 6, '2017-12-05 00:00:00': 4, '2018-01-10 00:00:00': 10, '2003-07-31 00:00:00': 2, '2013-03-23 00:00:00': 2, '2007-11-19 00:00:00': 2, '2016-10-30 00:00:00': 21, '2017-12-23 00:00:00': 11, 

In [8]:
# удаление бесполезных столбцов

train = train.drop(['issue_date', 'listing_date', 'pet_id'], axis = 1) 
train

,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,2.00000,Brown Tabby,0.80,7.78,13,9,0.0,1
1,1.00000,White,0.72,14.19,13,9,0.0,2
2,0.88339,Brown,0.15,40.90,15,4,2.0,4
3,1.00000,White,0.62,17.82,0,1,0.0,2
4,2.00000,Black,0.50,11.06,18,4,0.0,1
...,...,...,...,...,...,...,...,...
18829,2.00000,Tricolor,0.44,27.36,0,1,0.0,2
18830,0.88339,Brown,0.73,14.25,15,4,2.0,4
18831,0.00000,Calico Point,0.99,28.13,13,9,1.0,1
18832,0.00000,Tan,0.55,44.82,13,9,1.0,2


In [10]:
labelencoder = LabelEncoder()

train['color_type'] = labelencoder.fit_transform(train['color_type'])  # преобразование категориальных признаков в числовые
train

,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,2.00000,18,0.80,7.78,13,9,0.0,1
1,1.00000,53,0.72,14.19,13,9,0.0,2
2,0.88339,15,0.15,40.90,15,4,2.0,4
3,1.00000,53,0.62,17.82,0,1,0.0,2
4,2.00000,2,0.50,11.06,18,4,0.0,1
...,...,...,...,...,...,...,...,...
18829,2.00000,52,0.44,27.36,0,1,0.0,2
18830,0.88339,15,0.73,14.25,15,4,2.0,4
18831,0.00000,22,0.99,28.13,13,9,1.0,1
18832,0.00000,48,0.55,44.82,13,9,1.0,2


In [11]:
X = train.drop(['pet_category'], axis = 1)
y = train['pet_category']

In [12]:
def metriki(y_test, y_pred):
  cm = confusion_matrix(y_test, y_pred)

  TP = np.diag(cm)
  FP = np.sum(cm, axis=0) - TP
  FN = np.sum(cm, axis=1) - TP
  num_classes = 4
  TN = []
  for i in range(num_classes):
    temp = np.delete(cm, i, 0)   
    temp = np.delete(temp, i, 1)  
    TN.append(sum(sum(temp)))

  accuracy = (TP + TN) / (TP + TN + FP + FN)
  recall = TP / (TP + FN)
  precision = TP / (TP + FP)
  f1_score = 2 * precision * recall / (precision + recall)
  print()
  print('confusion_matrix: \n', confusion_matrix(y_test, y_pred))
  print()
  print('accuracy:', accuracy)
  print('recall:', recall)
  print('precision:', precision)
  print('f1_score:', f1_score)

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=1/3, random_state=42)

model = LogisticRegression() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(model.score(X_test, y_test)) # 69% неплохо, неплохо

metriki(y_test, y_pred)

# логистическая регрессия 
# хороший результат

0.6973558458107677

confusion_matrix: 
 [[   0    5   12   15]
 [   0 1218 1096   33]
 [   0  624 2955    7]
 [   0   63   45  205]]

accuracy: [0.99490284 0.70993947 0.71583307 0.97403632]
recall: [0.         0.51896037 0.82403793 0.65495208]
precision: [       nan 0.63769634 0.71932814 0.78846154]
f1_score: [       nan 0.57223397 0.76813101 0.71553229]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [14]:
sc = MinMaxScaler()
x_nm = sc.fit_transform(X)

X_train,X_test,y_train,y_test = train_test_split(x_nm, y, test_size=1/3, random_state=42)

model = LogisticRegression() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(model.score(X_test, y_test)) # 70% вполне непллохо

metriki(y_test, y_pred)

# логистическая регрессия с нормализованными данными дала очень хорошие результаты

# каждое значение списков accuracy, recall, precision и f1_score соответствует каждому классу
# т.к. их 4, то и 4 значения в каждом списке

0.7011787193373686

confusion_matrix: 
 [[   0    5   10   17]
 [   0 1245 1076   26]
 [   0  649 2930    7]
 [   0   47   39  227]]

accuracy: [0.99490284 0.71280663 0.71631093 0.97833705]
recall: [0.         0.53046442 0.81706637 0.72523962]
precision: [       nan 0.6397739  0.72256473 0.81949458]
f1_score: [       nan 0.58001398 0.76691533 0.76949153]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [15]:
x = StandardScaler().fit_transform(train) 

X_train,X_test,y_train,y_test = train_test_split(x, y, test_size=1/3, random_state=42)

model = LogisticRegression() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(model.score(X_test, y_test)) # 100% очень сильно

metriki(y_test, y_pred)

# самая точная модель - логистическая регрессия с стандартизированными данными
# т.к. FP и FN равны 0, то все параметры стали равными 1
# получается все супер идеально))

1.0

confusion_matrix: 
 [[  32    0    0    0]
 [   0 2347    0    0]
 [   0    0 3586    0]
 [   0    0    0  313]]

accuracy: [1. 1. 1. 1.]
recall: [1. 1. 1. 1.]
precision: [1. 1. 1. 1.]
f1_score: [1. 1. 1. 1.]


In [37]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=1/3, random_state=42)

score = []
for i in range(1,15):
    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    answers = knn.predict(X_test)
    score.append(knn.score(X_test,y_test))

print(score) # 77-80% 
print(np.mean(score)) # 79% хорошо

knn = KNeighborsClassifier(5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

metriki(y_test, y_pred)

# метод ближайших соседей, в сравнении, показал вполне хороший результат

0.7933600327674873

confusion_matrix: 
 [[   0    9   16    7]
 [   2 1753  581   11]
 [   0  484 3094    8]
 [   1   41   71  200]]

accuracy: [0.99442498 0.82032494 0.81522778 0.97785919]
recall: [0.         0.74691095 0.86279978 0.63897764]
precision: [0.         0.76650634 0.82243488 0.88495575]
f1_score: [       nan 0.75658179 0.84213391 0.74211503]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [32]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=1/3, random_state=42)

tree = DecisionTreeClassifier(criterion='gini', max_depth=15, random_state=1)

tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
print(tree.score(X_test, y_test))

metriki(y_test, y_pred_tree)

# дерево решений показало очень неплохие результаты

0.8517043644472762

confusion_matrix: 
 [[   9    6    5   12]
 [   6 1844  474   23]
 [   7  279 3272   28]
 [   4   43   44  222]]

accuracy: [0.99362854 0.867633   0.86667729 0.97546989]
recall: [0.28125    0.78568385 0.91243726 0.70926518]
precision: [0.34615385 0.84898711 0.86218709 0.77894737]
f1_score: [0.31034483 0.81610976 0.88660073 0.74247492]


In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
tree = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=1)
cvs = cross_val_score(estimator=tree, X=X, y=y, cv=kf, scoring='accuracy')

print(cvs)
print(cvs.mean()) # 

# дерево решений с кроссвалидацией 
# получилось хуже, чем дерево без кроссвалидации

[0.76187948 0.76479958 0.75657022 0.75418105 0.75305364]
0.7580967907426499


In [33]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=1/3, random_state=42)

# svm = SVC(kernel='linear', C=1.0, random_state=1)
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print(svm.score(X_test, y_test)) # 67% на фоне других так себе

metriki(y_test, y_pred_svm)


# метод опорных векторов работает с переменным успехом, самый низкий 
# что можно заметить в списке accuracy 

0.6784007645747053

confusion_matrix: 
 [[   0    7   21    4]
 [   0 1163 1156   28]
 [   0  590 2992    4]
 [   0  138   71  104]]

accuracy: [0.99490284 0.6943294  0.70659446 0.96097483]
recall: [0.         0.4955262  0.83435583 0.33226837]
precision: [       nan 0.61275026 0.70566038 0.74285714]
f1_score: [       nan 0.54793875 0.76463072 0.45916115]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
